# Forward in EditModel

In [1]:
from os import getcwd, chdir

if getcwd().endswith('notebooks'):
    chdir('..')

In [2]:
from data.getDataset import getCognatesSet
from data.datapipes import trainingDataPipe
from data.vocab import wordsToOneHots
from models.models import InferenceData
from torchdata.datapipes.iter import IterableWrapper


with open(f'./recons_data/iteration3_{str(1)}.txt', 'r', encoding='utf-8') as file:
    iteration = IterableWrapper([[line[:-1]] for line in file.readlines()[:-1]]).map(wordsToOneHots)

cognates = {}
for modernLanguage in ('french', 'spanish', 'portuguese', 'romanian', 'italian'):
    with open(f"./recons_data/data/{modernLanguage.capitalize()}_ipa.txt", "r", encoding="utf-8") as file:
        cognates[modernLanguage] = [line[1:-1] for line in file.readlines()[:-1]]
cognates = IterableWrapper([{l:cognates[l][i] for l in cognates} for i in range(len(cognates['french']))])
dp = trainingDataPipe(iteration, cognates, 5) #type: ignore
dp = iter(dp)

In [3]:
inpt = next(dp)
print(inpt[0][0].size())
print(inpt[0][1].size())

torch.Size([13, 5, 1])
torch.Size([5, 1])


In [4]:
from Source.reconstructionModel import ReconstructionModel
from data.vocab import vocabulary, PADDING_TOKEN, EOS_TOKEN
print("taille du vocabulaire :", len(vocabulary))
print("nombre de caractères de l'IPA :", len(vocabulary)-3)
print(vocabulary[PADDING_TOKEN], vocabulary[EOS_TOKEN])
model = ReconstructionModel(['french'], vocabulary, 50, 100)

taille du vocabulaire : 60
nombre de caractères de l'IPA : 57
59 57


In [5]:
from data.vocab import oneHotsToWords

print("x")
print(oneHotsToWords(inpt[0][0].squeeze(-1))[1])
print(inpt[0][0].squeeze(-1)[:,1])
print("\ny")
print(oneHotsToWords(inpt[1]["french"][0])[1])
print(inpt[1]["french"][0][:,1])

x
(altɨʁar)----
tensor([58,  0,  9, 16, 36, 41,  0, 14, 57, 59, 59, 59, 59], dtype=torch.int32)

y
(alteʁˈe------
tensor([58,  0,  9, 16,  3, 41, 51,  3, 59, 59, 59, 59, 59, 59],
       dtype=torch.int32)


In [6]:
rep = model.infer(*inpt)
print(rep[0].size(), rep[1].size())

torch.Size([12, 14, 5, 1, 2]) torch.Size([12, 14, 5, 1, 2])


In [9]:
print(rep[0][:, :-6, 1, 0, 0])
print(rep[0][:, :-6, 1, 0, 1])

tensor([[-3.7849, -4.2557, -3.9709, -4.1836, -4.0240, -4.0060, -4.0912,  0.0000],
        [-3.6969, -4.1889, -3.9078, -4.1734, -4.0366, -3.9087, -4.0807,  0.0000],
        [-3.8137, -4.1876, -3.9605, -4.2160, -4.0595, -4.0059, -4.1231,  0.0000],
        [-3.8136, -4.2289, -4.1281, -4.2282, -4.1282, -3.9280, -4.1357,  0.0000],
        [-3.7102, -4.2674, -4.0330, -4.0076, -4.2235, -3.8869, -3.9149,  0.0000],
        [-3.6319, -4.2581, -3.9692, -4.0574, -4.1379, -3.9190, -3.9654,  0.0000],
        [-3.7036, -4.1773, -3.9079, -4.0800, -4.1304, -3.9337, -3.9879,  0.0000],
        [-3.8137, -4.1863, -4.0731, -4.0555, -4.1502, -4.0541, -3.9640,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
       grad_fn=